Adding latitude and longitude geolocation to the 2020PB database

In [37]:
import pandas as pd
import numpy as np
from google.colab import files

[2020 Police Brutality Repo](https://github.com/2020PB/police-brutality)

[US Cities Database](https://github.com/kelvins/US-Cities-Database)




In [19]:
# police brutality data
pb_csv = pd.read_csv('https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations.csv')
pb_df = pd.DataFrame(pb_csv)

In [20]:
# location data
loc_csv = pd.read_csv('https://raw.githubusercontent.com/kelvins/US-Cities-Database/master/csv/us_cities.csv')
loc_df = pd.DataFrame(loc_csv)

In [21]:
print(pb_df.shape)
pb_df.info()

(932, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932 entries, 0 to 931
Data columns (total 27 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   state      932 non-null    object
 1   edit_at    932 non-null    object
 2   city       931 non-null    object
 3   name       932 non-null    object
 4   date       928 non-null    object
 5   date_text  932 non-null    object
 6   id         932 non-null    object
 7   Link 1     932 non-null    object
 8   Link 2     540 non-null    object
 9   Link 3     309 non-null    object
 10  Link 4     180 non-null    object
 11  Link 5     108 non-null    object
 12  Link 6     72 non-null     object
 13  Link 7     43 non-null     object
 14  Link 8     38 non-null     object
 15  Link 9     24 non-null     object
 16  Link 10    20 non-null     object
 17  Link 11    18 non-null     object
 18  Link 12    13 non-null     object
 19  Link 13    9 non-null      object
 20  Link 14    8 non-null 

In [22]:
print(loc_df.shape)
loc_df.info()

(29880, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29880 entries, 0 to 29879
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          29880 non-null  int64  
 1   STATE_CODE  29880 non-null  object 
 2   STATE_NAME  29880 non-null  object 
 3   CITY        29880 non-null  object 
 4   COUNTY      29858 non-null  object 
 5   LATITUDE    29880 non-null  float64
 6   LONGITUDE   29880 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.6+ MB


In [23]:
loc_df.head()

,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE
0,1,AK,Alaska,Adak,Aleutians West,55.999722,-161.207778
1,2,AK,Alaska,Akiachak,Bethel,60.891854,-161.392330
2,3,AK,Alaska,Akiak,Bethel,60.890632,-161.199325
3,4,AK,Alaska,Akutan,Aleutians East,54.143012,-165.785368
4,5,AK,Alaska,Alakanuk,Wade Hampton,62.746967,-164.602280


In [24]:
# drop redundant id column in loc_df
loc_df = loc_df.drop('ID', axis=1)

In [25]:
loc_df.head()

,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE
0,AK,Alaska,Adak,Aleutians West,55.999722,-161.207778
1,AK,Alaska,Akiachak,Bethel,60.891854,-161.392330
2,AK,Alaska,Akiak,Bethel,60.890632,-161.199325
3,AK,Alaska,Akutan,Aleutians East,54.143012,-165.785368
4,AK,Alaska,Alakanuk,Wade Hampton,62.746967,-164.602280


In [26]:
pb_df.head()

,state,edit_at,city,name,date,date_text,id,Link 1,Link 2,Link 3,Link 4,Link 5,Link 6,Link 7,Link 8,Link 9,Link 10,Link 11,Link 12,Link 13,Link 14,Link 15,Link 16,Link 17,Link 18,Link 19,Link 20
0,Unknown Location,https://github.com/2020PB/police-brutality/blo...,NaN,Compilation of incidents,2020-05-31,May 31st,tbd-tbd-1,https://twitter.com/perfectlyg0lden/status/126...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Vermont,https://github.com/2020PB/police-brutality/blo...,St. Johnsbury,Police shove a protesting woman down a set of ...,2020-06-03,June 3rd,vt-stjohnsbury-1,https://www.facebook.com/story.php?story_fbid=...,https://vtdigger.org/2020/06/03/police-arrest-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Louisiana,https://github.com/2020PB/police-brutality/blo...,New Orleans,Police throw tear-gas at protesters on a bridge.,2020-06-03,June 3rd,la-neworleans-1,https://twitter.com/misaacstein/status/1268381...,https://twitter.com/ckm_news/status/1268382403...,https://twitter.com/brynstole/status/126838134...,https://twitter.com/xxnthe/status/126842775987...,https://twitter.com/greg_doucette/status/12685...,https://www.wdsu.com/article/protesters-on-i-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Texas,https://github.com/2020PB/police-brutality/blo...,Austin,Police assault protesters,2020-05-30,May 30th,tx-austin-2,https://gfycat.com/tautimaginativedore,https://www.reddit.com/r/2020PoliceBrutality/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Texas,https://github.com/2020PB/police-brutality/blo...,Austin,Police shoot non-violent protester in the head,2020-05-30,May 30th,tx-austin-3,https://www.reddit.com/r/PublicFreakout/commen...,https://www.instagram.com/p/CA6TCIGnuWm/,https://www.youtube.com/watch?v=-BGyTi-KdKc (a...,https://streamable.com/o1uqgy (aftermath),https://cbsaustin.com/news/local/austin-teen-h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# rename pb_df columns
pb_df.rename(columns={'state':'STATE_NAME', 'city':'CITY'}, inplace=True)

In [31]:
# drop NaNs
pb_df.dropna(subset = ['CITY'], inplace=True)

In [32]:
pb_df.head()

,STATE_NAME,edit_at,CITY,name,date,date_text,id,Link 1,Link 2,Link 3,Link 4,Link 5,Link 6,Link 7,Link 8,Link 9,Link 10,Link 11,Link 12,Link 13,Link 14,Link 15,Link 16,Link 17,Link 18,Link 19,Link 20
1,Vermont,https://github.com/2020PB/police-brutality/blo...,St. Johnsbury,Police shove a protesting woman down a set of ...,2020-06-03,June 3rd,vt-stjohnsbury-1,https://www.facebook.com/story.php?story_fbid=...,https://vtdigger.org/2020/06/03/police-arrest-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Louisiana,https://github.com/2020PB/police-brutality/blo...,New Orleans,Police throw tear-gas at protesters on a bridge.,2020-06-03,June 3rd,la-neworleans-1,https://twitter.com/misaacstein/status/1268381...,https://twitter.com/ckm_news/status/1268382403...,https://twitter.com/brynstole/status/126838134...,https://twitter.com/xxnthe/status/126842775987...,https://twitter.com/greg_doucette/status/12685...,https://www.wdsu.com/article/protesters-on-i-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Texas,https://github.com/2020PB/police-brutality/blo...,Austin,Police assault protesters,2020-05-30,May 30th,tx-austin-2,https://gfycat.com/tautimaginativedore,https://www.reddit.com/r/2020PoliceBrutality/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Texas,https://github.com/2020PB/police-brutality/blo...,Austin,Police shoot non-violent protester in the head,2020-05-30,May 30th,tx-austin-3,https://www.reddit.com/r/PublicFreakout/commen...,https://www.instagram.com/p/CA6TCIGnuWm/,https://www.youtube.com/watch?v=-BGyTi-KdKc (a...,https://streamable.com/o1uqgy (aftermath),https://cbsaustin.com/news/local/austin-teen-h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Texas,https://github.com/2020PB/police-brutality/blo...,Austin,Police use tear gas & rubber bullets on protes...,2020-05-30,May 30th,tx-austin-4,https://twitter.com/highsettler/status/1266876...,https://imgur.com/gallery/VKuIIvu,https://gfycat.com/weirdthankfulgnatcatcher,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# merge the two dfs
incident_df = pb_df.merge(loc_df, how='inner')

In [36]:
incident_df.head()

,STATE_NAME,edit_at,CITY,name,date,date_text,id,Link 1,Link 2,Link 3,Link 4,Link 5,Link 6,Link 7,Link 8,Link 9,Link 10,Link 11,Link 12,Link 13,Link 14,Link 15,Link 16,Link 17,Link 18,Link 19,Link 20,STATE_CODE,COUNTY,LATITUDE,LONGITUDE
0,Louisiana,https://github.com/2020PB/police-brutality/blo...,New Orleans,Police throw tear-gas at protesters on a bridge.,2020-06-03,June 3rd,la-neworleans-1,https://twitter.com/misaacstein/status/1268381...,https://twitter.com/ckm_news/status/1268382403...,https://twitter.com/brynstole/status/126838134...,https://twitter.com/xxnthe/status/126842775987...,https://twitter.com/greg_doucette/status/12685...,https://www.wdsu.com/article/protesters-on-i-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LA,Jefferson,29.963071,-90.160953
1,Louisiana,https://github.com/2020PB/police-brutality/blo...,New Orleans,Police throw tear-gas at protesters on a bridge.,2020-06-03,June 3rd,la-neworleans-1,https://twitter.com/misaacstein/status/1268381...,https://twitter.com/ckm_news/status/1268382403...,https://twitter.com/brynstole/status/126838134...,https://twitter.com/xxnthe/status/126842775987...,https://twitter.com/greg_doucette/status/12685...,https://www.wdsu.com/article/protesters-on-i-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LA,Orleans,29.960484,-90.075301
2,Texas,https://github.com/2020PB/police-brutality/blo...,Austin,Police assault protesters,2020-05-30,May 30th,tx-austin-2,https://gfycat.com/tautimaginativedore,https://www.reddit.com/r/2020PoliceBrutality/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TX,Hays,30.210692,-97.942749
3,Texas,https://github.com/2020PB/police-brutality/blo...,Austin,Police assault protesters,2020-05-30,May 30th,tx-austin-2,https://gfycat.com/tautimaginativedore,https://www.reddit.com/r/2020PoliceBrutality/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TX,Travis,30.230300,-97.714400
4,Texas,https://github.com/2020PB/police-brutality/blo...,Austin,Police assault protesters,2020-05-30,May 30th,tx-austin-2,https://gfycat.com/tautimaginativedore,https://www.reddit.com/r/2020PoliceBrutality/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TX,Williamson,30.505972,-97.747187


In [39]:
# saving the dataframe 
incident_df.to_csv('pv_incidents.csv') 

In [40]:
files.download('pv_incidents.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>